# Deep Learning for Tree crown detection 

### Final project - MET 4:  'Advances Programming for Remote Sensing and GIS'
### by: Hyeonmin Kang, Prince Lawson, Svenja Dobelmann 

#### Description: 

Tree crown image (*WingtraUniwald.tiff*) is taken from Lidar and UAV data (See script 'modelling.ipynb'). Tree crown polygons (*treeCrowns1.shp*) were generated by Watershed method in R Studio. 

This script created more than 3800 rectangular raster images and masks. At first, we found the center of the tree crown polygons and buffered them in rectangular form. 
And we cut images in 256x256 pixel size using rasterio window. After that, we masked the tree crowns. 
Tree crowns are 1, and the background is 0.

## Install packages 

* geopandas : https://geopandas.org/en/stable/getting_started/install.html
* numpy: https://numpy.org/install/
* matplotlib.pyplot: https://matplotlib.org/stable/users/installing/index.html
* rasterio: https://rasterio.readthedocs.io/en/latest/installation.html 
* shapely: https://shapely.readthedocs.io/en/latest/installation.html
* fiona : !pip install Fiona
* json: !pip install jsons


## Import packages and set parameters

In [12]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
import os
from itertools import product
import rasterio as rio
from rasterio import windows
from shapely import wkt
import fiona
import rasterio.mask
import json
import sys

#### the whole tree crown raster image and shapefile
large_image_name = 'WingtraUniwald.tiff'
large_tree_poly_name = 'treeCrowns1.shp'
patch_width = 256 
patch_height = 256
SIZE = 256

### Create directory names and create folders

In [13]:
#### path for the raster images and masks
training_img_path = 'training_img/'
training_img_256_path = 'training_img_256/' 
mask_img_path = 'mask_img/'
mask_256_path = 'mask_img_256/'

# Create directories 
os.makedirs(training_img_path, exist_ok = True)
os.makedirs(training_img_256_path, exist_ok = True)
os.makedirs(mask_img_path, exist_ok = True)
os.makedirs(mask_256_path, exist_ok = True)

## Function to clip images and maks in 256 x 256 pixel sizes. 

In [14]:
# Clip the images and masks in 256 x 256 pixel sizes. 
# We can keep all the geospatial information of images by using rasterio. 

def get_patches(img, width=patch_width, height=patch_height):
    ncols, nrows = img.meta['width'], img.meta['height']
    offsets = product(range(0, ncols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=ncols, height=nrows)
    for col_off, row_off in offsets:
        window =windows.Window(col_off=col_off, 
                               row_off=row_off, 
                               width=width, 
                               height=height).intersection(big_window)
        transform = windows.transform(window, img.transform)
        yield window, transform

### Load polygon and create dataframe of centroids of polygons and their buffers

In [15]:
# Load the tree crown shapefile (whole polygons in one file)
crowns = gpd.read_file(large_tree_poly_name)
# Create a column for center of polygon 
crowns['center'] = crowns.geometry.centroid
# Create a column for rectangular buffer of center of polygon.  
crowns['buffer'] = crowns.center.buffer(10, cap_style = 3)

### Here we create the rasters of tree crowns which are buffered from the center. They are cut in the size of 256 x 256 pixel.

In [16]:
%%time

only_buffer= crowns[['buffer']]

for index, row in only_buffer.iterrows():
    # create geopandas dataframe including only buffer geometries
    poly_df = gpd.GeoDataFrame(geometry = row, crs="EPSG:32632")
    # not insert index into dataframe columns
    poly_df = poly_df.reset_index(drop=True)
    # json form is the one of the required and easiest form to create masks by rasterio.mask.mask
    poly_json = poly_df.to_json()
    poly_json = json.loads(poly_json)
    poly_json = poly_json['features'][0]['geometry']
    # create an empty list
    shape = []
    # save all the geometries of buffers
    shape.append(poly_json)
    # clip the buffer geometries from the whole large raster file of tree crowns
    # and keep all geospatial information
    with rasterio.open(large_image_name) as src:
        out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
        out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})
    outpath = os.path.join(training_img_path,f'tr_img_{int(index)}.tif') 
    # save them in the folder
    with rasterio.open(outpath, "w", **out_meta) as dest:
        dest.write(out_image)   
    # open them again
    with rasterio.open(outpath) as large_img:
        meta = large_img.meta.copy()
        # use rasterio.window and clip them in 256 x 256 size
        for window, transform in get_patches(large_img):
          # the images in other sizes will not be saved
            if window.height != SIZE or window.width != SIZE:
                continue
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            outpath2 = os.path.join(training_img_256_path,f'tr_img_256_{int(index)}.tif')
            with rasterio.open(outpath2, 'w', **meta) as final_img:
                final_img.write(large_img.read(window=window))

CPU times: user 3min 45s, sys: 18.4 s, total: 4min 4s
Wall time: 4min 5s


### Same process for masks

In [18]:
%%time

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")




polygon= crowns[['geometry']]
for index, row in polygon.iterrows():
    geom_value = (row[0],index)
    raster_path = os.path.join(training_img_path,f'tr_img_{int(index)}.tif')
    # Rasterize vector 
    # only tree crowns will have value and the rest of them (as background) will be 0 
    with rasterio.open(raster_path, 'r') as img_raster:
        rasterized = features.rasterize(geom_value,
                                    out_shape = img_raster.shape, 
                                    fill = 0, #background value
                                    out = None, 
                                    transform = img_raster.transform, 
                                    all_touched = False,
                                    merge_alg = MergeAlg.replace,
                                    dtype = np.int16)
        
        mask_path = os.path.join(mask_img_path ,f'msk_img_{int(index)}.tif')
        # keep the geospatial information of raster for saving masks 
        output_profile = img_raster.profile
        # we need only one band and int16 is enough. 
        output_profile.update(
              count=1,
              dtype=rasterio.int16
        )
        # save the masks (but the pixel size is not 256 x 256 yet)
        with rasterio.open(mask_path, 'w', **output_profile) as dst:
            dst.write(rasterized,1)
        # reopen the images
        with rasterio.open(mask_path) as large_img:
            meta = large_img.meta.copy() 
            # clip them again like above code. 
            for window, transform in get_patches(large_img):
                if window.height != SIZE or window.width != SIZE:
                    continue
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath2 = os.path.join(mask_256_path ,f'msk_img_256_{int(index)}.tif')
                with rasterio.open(outpath2, 'w', **meta) as final_img:
                    final_img.write(large_img.read(window=window))

CPU times: user 14.2 s, sys: 8.57 s, total: 22.8 s
Wall time: 23.2 s
